# Project Pipeline
Execute the cells step by step to obtain a prediction and score for your configuration.
In order to make the widgets work you might need to execute 
```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```
on your system.

## 1. Imports

In [81]:
import importlib
import data_utils
import widget_ui as ui

# Load potential changes in the modules during development
data_utils = importlib.reload(data_utils)
ui = importlib.reload(ui)

## 2. Load Data Sets
Load training and test data. Furthermore, create a sample set for quickly calculating transformations such as PCA.

In [82]:
data = data_utils.load_from('data/all_train.csv', 'data/all_test.csv')
sample_data = data_utils.load_sample_data('data/all_sample.csv')

## 3. Preprocessing

In [83]:
preprocessors = {}

### 3.1 Remove Correlations
Choose between two methods to remove correlations: Pearson correlation coefficient or PCA.

__Pearson correlation coefficient:__ Find correlated pairs of features and remove one feature of these two, respectively. All pairs with a Pearson correlation coefficient above the selected threshold are considered correlated.

__PCA:__ Apply a principal component analysis. The number of principal components can be choosen.

In [84]:
ui.preprocessors_ui(preprocessors, sample_data)

## 4. Configure Classifiers

In [85]:
classifiers = {}

### 4.1 Multilayer Perceptron

In [86]:
ui.mlp_ui(classifiers)

### 4.2 Naive Bayes

In [87]:
ui.nb_ui(classifiers)

## 5. Training

In [88]:
ui.training_ui(data, sample_data, preprocessors, classifiers)

## 6. Prediction

In [89]:
ui.prediction_ui(data, sample_data, preprocessors, classifiers)